In [12]:
from Strategies import Strategy
import random

---

### Basic

In [8]:
class Assessment:
    def __init__(self, questions: list[Strategy]):
        random.shuffle(questions)

        self._questions = questions

    def start(self) -> bool:
        result = map(lambda question: question.doAlgorithm(), self._questions)
        
        return all(result)

### Factory

In [ ]:
class AssessmentFactory(object):
    @staticmethod
    def createAdditionAssessment(params: list[tuple[int, int, str]]) -> Assessment:
        def createQuestion(x: int, y: int, question: str) -> StrategyContext:
            strategy = AdditionStrategy[int]((x, y))
            strategyContext = SurveyContextStrategyFactory.create(strategy, question)

            return strategyContext

        questions = [createQuestion(x, y, question) for (x, y, question) in params]

        return Assessment(questions)

    @staticmethod
    def createMultiplyAssessment(numbers: list, count: int = 0) -> Assessment:
        pass
        # def getQuestion() -> StrategyContext:
        #    x, y = getItemsRandomly(numbers, 2)

        #    strategy = MultiplyStrategy((x, y))
        #    return SurveyContextStrategyFactory.create(strategy)

        # return AssessmentFactory.create(getQuestion, count)